# Part 4. Test the word error rate

In [1]:
import numpy as np
import pandas as pd
import datasets
from datasets import load_dataset
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
from transformers import Wav2Vec2ProcessorWithLM
import torch
from jiwer import wer

In [2]:
from datasets import load_from_disk
#our data in currently, in encoded formated
#so i will use preporcessed features and clean text data
urdudata = load_from_disk("cache/saved")

In [3]:
test_text=pd.read_csv('tex-test.csv')
test_text.head()

,text
0,یہی تناسب یوتھ کا بھی ہے
1,آیی ایم ایف کے ساتھ کن شرایط پر بات ہو رہی ہے
2,اور پھر سپاہی سے کہا ارے ہاں
3,اب صرف اور صرف انتظار ہے اگلے ماہ
4,تو میں پی ٹی وی میں اینکر تھا


# Without LM model

In [ ]:

def map_to_pred(batch):
    input_values = processor(batch["input_values"], sampling_rate =16000,
                             return_tensors="pt", padding="longest").input_values
    with torch.no_grad():
        logits = model(input_values.to("cuda")).logits
    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.batch_decode(predicted_ids)
    batch["transcription"] = transcription
    return batch

In [ ]:
model = Wav2Vec2ForCTC.from_pretrained('urdumodel',local_files_only=True).to("cuda")
processor = Wav2Vec2Processor.from_pretrained('urdumodel',local_files_only=True)

In [ ]:
result = urdudata['test'].map(map_to_pred, batched=True, batch_size=12, 
                              remove_columns=urdudata['test'].column_names)

print("WER:", wer(test_text['text'].to_list(), result["transcription"]))


WER: 0.2720071796472608

# With Language Model

#now here we need to remove some tokens, because our LM and vocaib tokens did not match  
visit this link for info
https://github.com/huggingface/transformers/issues/15392  
perform following steps to fix this  

### change special_tokens_map.json
from 
```
{
  "additional_special_tokens": [
    {
      "content": "<s>",
      "lstrip": false,
      "normalized": true,
      "rstrip": false,
      "single_word": false
    },
    {
      "content": "</s>",
      "lstrip": false,
      "normalized": true,
      "rstrip": false,
      "single_word": false
    }
  ],
  "bos_token": "<s>",
  "eos_token": "</s>",
  "pad_token": "[PAD]",
  "unk_token": "[UNK]"
}
```
to
```
{"bos_token": null, "eos_token": null, "unk_token": "<UNK>", "pad_token": "<PAD>"}
```
### changes in tokenizer_config.json
From 
```
"bos_token": "<s>",
"eos_token": "</s>",
```
To
```
"bos_token": null,
"eos_token": null,
```
Dont remove other lines

### changes in added_tokens.json
From
```
{"<s>": 48, "</s>": 49}
```
To
`{}`

### Chnages in  alphabet.json
Remove these two ```"<s>", "</s>"```


In [4]:
def map_to_pred(batch):
    input_values = processor(batch["input_values"], sampling_rate =16000,
                             return_tensors="pt", padding="longest").input_values
    with torch.no_grad():
        logits = model(input_values.to("cuda")).logits
    predicted_ids = torch.argmax(logits, dim=-2)
    transcription = processor.batch_decode(logits.cpu().numpy()).text
    batch["transcription"] = transcription
    return batch

In [5]:
model = Wav2Vec2ForCTC.from_pretrained('urdumodellm',local_files_only=True).to("cuda")
processor = Wav2Vec2ProcessorWithLM.from_pretrained("urdumodellm",local_files_only=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
result = urdudata['test'].map(map_to_pred, batched=True, batch_size=12, 
                              remove_columns=urdudata['test'].column_names)

print("WER:", wer(test_text['text'].to_list(), result["transcription"]))

  0%|          | 0/275 [00:00<?, ?ba/s]

2022-08-13 18:34:17.133532: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


WER: 0.22007179647260808
